In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from config import g_key

In [2]:
# Configure gmaps
gmaps.configure(api_key=g_key)


In [3]:
# MAKE SURE YOU SAVE THE EXCEL FILE AS UTF-8 CSV OR IT WILL NOT WORK
# OR YOU CAN CONVERT READER AS UTF
weather_data = pd.read_csv('weatherAPI_data_utf.csv')
weather_data

,city_id,city_name,country,latitude,longitude,weather_date,temp,humidity,cloudiness,wind_speed
0,4032243,Vaini,TO,-21.2000,-175.2000,11/13/2021,24.04,70,19,4.08
1,3366880,Hermanus,ZA,-34.4187,19.2345,11/13/2021,14.94,87,55,3.20
2,964420,Port Elizabeth,ZA,-33.9180,25.5701,11/13/2021,17.16,82,20,5.66
3,2264557,Ponta do Sol,PT,32.6667,-17.1000,11/13/2021,20.00,70,10,0.94
4,1794209,Simao,CN,22.7886,100.9748,11/13/2021,15.54,99,100,1.48
...,...,...,...,...,...,...,...,...,...,...
549,1788852,Xining,CN,36.6167,101.7667,11/13/2021,-1.08,49,58,1.27
550,2093846,Kiunga,PG,-6.1219,141.2906,11/13/2021,33.74,49,87,1.36
551,3354077,Opuwo,NaN,-18.0607,13.8400,11/13/2021,24.05,49,99,1.89
552,709584,Druzhba,UA,52.0428,33.9453,11/13/2021,-0.53,89,70,2.63


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# we want the 'latitude' column only
lat_col = weather_data['latitude']

# we want the 'longitude' column only 
lon_col = weather_data['longitude']

#combine the two columns as one data frame
# lat_lon_col = lat[lat_col,lon_col]
lat_lon_col = weather_data[['latitude', 'longitude']]
lat_lon_col


,latitude,longitude
0,-21.2000,-175.2000
1,-34.4187,19.2345
2,-33.9180,25.5701
3,32.6667,-17.1000
4,22.7886,100.9748
...,...,...
549,36.6167,101.7667
550,-6.1219,141.2906
551,-18.0607,13.8400
552,52.0428,33.9453


In [5]:
# we need 'humidity' column only now
humidity_col = weather_data['humidity']
humidity_col

0      70
1      87
2      82
3      70
4      99
       ..
549    49
550    49
551    49
552    89
553    36
Name: humidity, Length: 554, dtype: int64

In [6]:
# Plot the Heatmap
# layout = syntax {width: , height: , border:, padding: , margin: , etc.}
fig = gmaps.figure(layout={'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'},zoom_level=2,center=(15,25))

# This will be the heat layer
heat_area = gmaps.heatmap_layer(lat_lon_col, weights=humidity_col, dissipating=False, max_intensity=100, 
point_radius=1.5)

# Add heat layer
fig.add_layer(heat_area)

# display the 'fig'
fig



Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# We will set the temperature between 75 and 65
# It is up to you set the values... I find it fitting at these values
sixtyfive_seventyfive_temperature = (weather_data.temp < 26.6) & (weather_data.temp > 21.1)
twelve_windSpeed = (weather_data.wind_speed < 12)
zero_cloudiness = (weather_data.cloudiness == 0)
fitting_weather_criteria = sixtyfive_seventyfive_temperature & twelve_windSpeed & zero_cloudiness 


In [8]:
print("sixtyfive_seventyfive_temperature:")
print(sixtyfive_seventyfive_temperature)
print(" - " * 20)
print("twelve_windSpeed:")
print(twelve_windSpeed)
print(" - " * 20)
print("zero_cloudiness:")
print(zero_cloudiness)
print(" - " * 20)
print("fitting_weather_criteria:")
print(fitting_weather_criteria)

sixtyfive_seventyfive_temperature:
0       True
1      False
2      False
3      False
4      False
       ...  
549    False
550    False
551     True
552    False
553     True
Name: temp, Length: 554, dtype: bool
 -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 
twelve_windSpeed:
0      True
1      True
2      True
3      True
4      True
       ... 
549    True
550    True
551    True
552    True
553    True
Name: wind_speed, Length: 554, dtype: bool
 -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 
zero_cloudiness:
0      False
1      False
2      False
3      False
4      False
       ...  
549    False
550    False
551    False
552    False
553    False
Name: cloudiness, Length: 554, dtype: bool
 -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 
fitting_weather_criteria:
0      False
1      False
2      False
3      False
4      False
       ...  
549    False
550    False
551    False
552    False
553    False
Length: 554, dtype: bool


In [9]:
# All the values above are in boolean... so we must merge dataframes and apply the boolean into the dataFrame
# to get those true values only!
fitting_weather_data = weather_data[fitting_weather_criteria]

# Now all of the values in 'fitting_weather_data' is true and false values
# so we must drop all of the false values or garbage values
fitting_weather_data = fitting_weather_data.dropna()

# The index is all messed up since we dropped rows that were false... time to reset index
fitting_weather_data = fitting_weather_data.reset_index()

# print data frame
fitting_weather_data

,index,city_id,city_name,country,latitude,longitude,weather_date,temp,humidity,cloudiness,wind_speed
0,11,2413419,Gunjur,GM,13.2019,-16.7339,11/13/2021,25.04,88,0,2.06
1,61,371466,Leer,SS,8.3018,30.1418,11/13/2021,23.73,60,0,1.69
2,86,2070998,Geraldton,AU,-28.7667,114.6000,11/13/2021,23.23,64,0,6.17
3,101,1790437,Zhuhai,CN,22.2769,113.5678,11/13/2021,22.25,53,0,6.17
4,121,1090225,Koungou,YT,-12.7336,45.2042,11/13/2021,25.84,78,0,3.60
5,131,286621,Salalah,OM,17.0151,54.0924,11/13/2021,22.05,49,0,1.54
6,161,1819729,Hong Kong,HK,22.2855,114.1577,11/13/2021,22.40,54,0,1.79
7,199,2065594,Mount Isa,AU,-20.7333,139.5000,11/13/2021,23.87,22,0,7.20
8,230,2152659,Port Macquarie,AU,-31.4333,152.9167,11/13/2021,24.94,25,0,4.36
9,236,377472,Bentiu,SS,9.2333,29.8333,11/13/2021,24.56,59,0,1.27


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# hotel_df as fitting_weather_data data frame
hotel_df = fitting_weather_data

# Add a "Hotel Name" column to 'hotel_df
hotel_df['Hotel Name'] = ""
hotel_df

,index,city_id,city_name,country,latitude,longitude,weather_date,temp,humidity,cloudiness,wind_speed,Hotel Name
0,11,2413419,Gunjur,GM,13.2019,-16.7339,11/13/2021,25.04,88,0,2.06,
1,61,371466,Leer,SS,8.3018,30.1418,11/13/2021,23.73,60,0,1.69,
2,86,2070998,Geraldton,AU,-28.7667,114.6000,11/13/2021,23.23,64,0,6.17,
3,101,1790437,Zhuhai,CN,22.2769,113.5678,11/13/2021,22.25,53,0,6.17,
4,121,1090225,Koungou,YT,-12.7336,45.2042,11/13/2021,25.84,78,0,3.60,
5,131,286621,Salalah,OM,17.0151,54.0924,11/13/2021,22.05,49,0,1.54,
6,161,1819729,Hong Kong,HK,22.2855,114.1577,11/13/2021,22.40,54,0,1.79,
7,199,2065594,Mount Isa,AU,-20.7333,139.5000,11/13/2021,23.87,22,0,7.20,
8,230,2152659,Port Macquarie,AU,-31.4333,152.9167,11/13/2021,24.94,25,0,4.36,
9,236,377472,Bentiu,SS,9.2333,29.8333,11/13/2021,24.56,59,0,1.27,


In [11]:
# Use Params!
search_dict = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    # must find the latitude
    lat_find = row["latitude"]
    
    # find the longitude
    lng_find = row["longitude"]
    
    city_name = row["city_name"]

    search_dict["location"] = f"{lat_find},{lng_find}"
    
    # google_url for g_key
    google_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(google_url, params=search_dict)
    
    # convert data to json to see data
    name_address = name_address.json()
    
    # make sure you specify to be equals to 'results' since it prints out a dict of results
    name_address = name_address['results']
    # If data is missing, we will use the try method... if it fails, it will print what is missing
    try:
        name_success = name_address[0]["name"]
        hotel_df.loc[index, "Hotel Name"] = name_address[0]["name"]
        print(f'Closest hotel in {city_name} is {name_success}')
    except (KeyError, IndexError):
        print("Missing values! Skip! Skip!")

Closest hotel in Gunjur is Footsteps Eco Lodge, The Gambia
Missing values! Skip! Skip!
Closest hotel in Geraldton is Ocean Centre Hotel
Closest hotel in Zhuhai is Pullman Zhuhai
Closest hotel in Koungou is Hôtel Caribou
Closest hotel in Salalah is Muscat International Hotel plaza Salalah
Closest hotel in Hong Kong is Hotel ICON
Closest hotel in Mount Isa is Isa Hotel
Closest hotel in Port Macquarie is Port Macquarie Hotel
Closest hotel in Bentiu is Mandela hôtel
Closest hotel in Carnarvon is Carnarvon Motel
Closest hotel in Ballina is Ramada Hotel & Suites by Wyndham Ballina Byron
Closest hotel in Ormara is Alqadri hotel zero point ormara
Closest hotel in Pasni is Juddi Hotel
Closest hotel in Santana do Matos is Balneário Belazer
Missing values! Skip! Skip!
Closest hotel in Acapulco de Juárez is Hotel Krystal Beach Acapulco
Closest hotel in Yulara is Longitude 131
Closest hotel in Byron Bay is The Park Hotel Motel


#### CODE GIVEN

In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city_name}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…